# Second Order Markov Process Simulation (S4)

In [ ]:
import numpy as np
np.random.seed(0)
from gendata import genbasicseqs, plotprobs, genfirstorder,gendata
from ctw import CTW
import pickle

N=300000
randparams=True
num_trials=100
thresh=0.001
ks = [2,3,4]

numx=3
numy=3

def kl(d1,d2):
    kl = 0
    for i in range(len(d1)):
        kl += d1[i]*np.log2(d1[i]/d2[i])
    return kl

def avg_kl(px1,px2):
    if px1.shape != px2.shape:
        raise Exception
    N = px1.shape[1]
    tot = 0
    for n in range(N):
        tot += kl(px1[:,n],px2[:,n])
    return tot/N

def gen_seq(N,numx=3,numy=2):
    params ={
    '[[0,0],[0,0]]':None,
    '[[0,0],[0,1]]':None,
    '[[0,0],[0,2]]':None,
    '[[0,0],[1,0]]':None,
    '[[0,0],[1,1]]':None,
    '[[0,0],[1,2]]':None,
    '[[0,0],[2,0]]':None,
    '[[0,0],[2,1]]':None,
    '[[0,0],[2,2]]':None,
    '[[0,1],[0,0]]':None,
    '[[0,1],[0,1]]':None,
    '[[0,1],[0,2]]':None,
    '[[0,1],[1,0]]':None,
    '[[0,1],[1,1]]':None,
    '[[0,1],[1,2]]':None,
    '[[0,1],[2,0]]':None,
    '[[0,1],[2,1]]':None,
    '[[0,1],[2,2]]':None,
    '[[0,2],[0,0]]':None,
    '[[0,2],[0,1]]':None,
    '[[0,2],[0,2]]':None,
    '[[0,2],[1,0]]':None,
    '[[0,2],[1,1]]':None,
    '[[0,2],[1,2]]':None,
    '[[0,2],[2,0]]':None,
    '[[0,2],[2,1]]':None,
    '[[0,2],[2,2]]':None,
    '[[1,0],[0,0]]':None,
    '[[1,0],[0,1]]':None,
    '[[1,0],[0,2]]':None,
    '[[1,0],[1,0]]':None,
    '[[1,0],[1,1]]':None,
    '[[1,0],[1,2]]':None,
    '[[1,0],[2,0]]':None,
    '[[1,0],[2,1]]':None,
    '[[1,0],[2,2]]':None,
    '[[1,1],[0,0]]':None,
    '[[1,1],[0,1]]':None,
    '[[1,1],[0,2]]':None,
    '[[1,1],[1,0]]':None,
    '[[1,1],[1,1]]':None,
    '[[1,1],[1,2]]':None,
    '[[1,1],[2,0]]':None,
    '[[1,1],[2,1]]':None,
    '[[1,1],[2,2]]':None,
    '[[1,2],[0,0]]':None,
    '[[1,2],[0,1]]':None,
    '[[1,2],[0,2]]':None,
    '[[1,2],[1,0]]':None,
    '[[1,2],[1,1]]':None,
    '[[1,2],[1,2]]':None,
    '[[1,2],[2,0]]':None,
    '[[1,2],[2,1]]':None,
    '[[1,2],[2,2]]':None,
    '[[2,0],[0,0]]':None,
    '[[2,0],[0,1]]':None,
    '[[2,0],[0,2]]':None,
    '[[2,0],[1,0]]':None,
    '[[2,0],[1,1]]':None,
    '[[2,0],[1,2]]':None,
    '[[2,0],[2,0]]':None,
    '[[2,0],[2,1]]':None,
    '[[2,0],[2,2]]':None,
    '[[2,1],[0,0]]':None,
    '[[2,1],[0,1]]':None,
    '[[2,1],[0,2]]':None,
    '[[2,1],[1,0]]':None,
    '[[2,1],[1,1]]':None,
    '[[2,1],[1,2]]':None,
    '[[2,1],[2,0]]':None,
    '[[2,1],[2,1]]':None,
    '[[2,1],[2,2]]':None,
    '[[2,2],[0,0]]':None,
    '[[2,2],[0,1]]':None,
    '[[2,2],[0,2]]':None,
    '[[2,2],[1,0]]':None,
    '[[2,2],[1,1]]':None,
    '[[2,2],[1,2]]':None,
    '[[2,2],[2,0]]':None,
    '[[2,2],[2,1]]':None,
    '[[2,2],[2,2]]':None,
    }
    for key in params.keys():
        xparam = np.random.exponential(size=numx)
        xparam = xparam/sum(xparam)
        yparam = np.random.exponential(size=numy)
        yparam = yparam/sum(yparam)
        params[key]=(xparam,yparam)
    
    x,y = gendata(N=N,params=params,plot=False)
    return x,y


## (S4) : $Y\rightarrow Y$, $Y\rightarrow X$, $X\rightarrow X$, $X\rightarrow Y$ 

In [ ]:
TDIs = np.zeros((num_trials,len(ks)))
PDIs = np.zeros((num_trials,len(ks)))
for trial in range(num_trials):
    print('--------------- Trial %i ---------------'%(trial+1))
    
    x,y = gen_seq(N)

    ctw_full = CTW(depth=2,symbols=numx,sidesymbols=numy)
    px_full = ctw_full.predict_sequence(x,y)
    ks = [2,3,4]
    for k in ks:
        print('k=%i'%k)
        ctw_marg = CTW(depth=k,symbols=numx)
        px_marg = ctw_marg.predict_sequence(x)
        DIk = avg_kl(px_full[:,k-2:],px_marg)
        print('TDI = %1.3f'%DIk)
        ctw_part = CTW(depth=k+2,symbols=numx,sidesymbols=numy,staleness=k)
        px_part = ctw_part.predict_sequence(x,y)
        PDIk = avg_kl(px_full[:,k:],px_part)
        print('PDI = %1.3f'%PDIk)
        TDIs[trial,k-2]=DIk
        PDIs[trial,k-2]=PDIk
        if(DIk-PDIk < thresh):
            print('Converged!')
            break
            
pickle.dump((TDIs,PDIs), open('case4n%i.pickle'%num_trials, 'wb'))